In [9]:
import numpy as np
arr = np.array([['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines'],
 ['was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to'], 
 ['be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the'], 
 ['rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is'], 
 ['made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']])


In [10]:
arr1 = np.array([['from', 'guy', 'kuo', 'subject',  'clock', 'poll', 'final', 'summary', 'final', 'call', 'for', 'si', 'reports', 'keywords', 'acceleration', 'clock', 'upgrade', 'article', 'shelley', 'qvfo', 'innc', 'organization', 'university', 'of', 'washington', 'lines', 'nntp', 'posting', 'host', 'carson', 'washington', 'edu', 'fair', 'number', 'of', 'brave', 'souls', 'who', 'upgraded', 'their', 'si', 'clock', 'oscillator', 'have', 'shared', 'their', 'experiences', 'for', 'this', 'poll', 'please', 'send', 'brief', 'message', 'detailing', 'your', 'experiences', 'with', 'the', 'procedure', 'top', 'speed', 'attained', 'rated', 'speed', 'add', 'cards', 'adapters', 'heat', 'sinks', 'hour', 'of', 'usage', 'per', 'day', 'floppy', 'disk', 'functionality', 'with', 'and', 'floppies', 'are', 'especially', 'requested', 'will', 'be', 'summarizing', 'in', 'the', 'next', 'two', 'days', 'so', 'please', 'add', 'to', 'the', 'network', 'knowledge', 'base', 'if', 'you', 'have', 'done', 'the', 'clock', 'upgrade', 'and', 'havent', 'answered', 'this', 'poll', 'thanks', 'guy', 'kuo']])
arr1 = np.reshape(arr1,(5,23))

In [11]:
import pandas as pd
import gensim

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from pprint import pprint

import gensim.corpora as corpora
from gensim.corpora.dictionary import Dictionary

from gensim.models import CoherenceModel
from pprint import pprint


class fit_lda:
    
    # To convert the 2d array data into list
    def convert_to_list(self,inp1, inp2):
        data_words1 = inp1.tolist()
        data_words2= inp2.tolist()
        return data_words1,data_words2
    
    
    #To create corpus and id2word
    def data_processing(self,data_words):
        id2word = corpora.Dictionary(data_words)
        texts = data_words
        corpus = [id2word.doc2bow(text) for text in texts]
        return corpus,id2word
    
    
    # To fit the LDA model
    def Lda_fit(self,data_words,corpus1,id2word1,n,r,c,p):
        lda_model = gensim.models.LdaMulticore(corpus=corpus1,
                                       id2word=id2word1,
                                       num_topics=n, 
                                       random_state=r,
                                       chunksize=c,
                                       passes=p,
                                       per_word_topics=True)
        pprint(lda_model.print_topics())
        return lda_model
    
    
    # To find the coherence score
    def coherence(self,data_words,id2word1,modelfit):
        coherence_model_lda = CoherenceModel(model=modelfit, texts=data_words, dictionary=id2word1, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        print('*****************************************************************************************************************************')
        print('\nCoherence Score of the trained LDA model : ', coherence_lda)
        print('\n*****************************************************************************************************************************')
    
    
    #To compute the coherence score of different LDA model obtained from multiple number of topics
    def compute_coherence_values(self,data_words,corpus1,id2word1):
        limit=50; start=2; step=7;
        coherence_values = []
        model_list = []
        for num_topics in range(start, limit, step):
          model=gensim.models.LdaMulticore(corpus=corpus1, id2word=id2word1, num_topics=num_topics)
          model_list.append(model)
          coherencemodel = CoherenceModel(model=model, texts=data_words, dictionary=id2word1, coherence='c_v')
          coherence_values.append(coherencemodel.get_coherence())
       
        return model_list, coherence_values
    
    
    # To find the best coherence score and optimal number of topics for the best coherence score of the best LDA model
    def best_model_lda(self,data_words,model_list, coherence_values):
        range_topics = range(2, 50, 7)
        best_coherence= max(coherence_values)  
        Topic_best_coherence = range_topics[coherence_values.index(best_coherence)] 
        print("\nCoherence score of the best LDA model : ", best_coherence)
        print("Optimal number of topics of the best LDA model : ", Topic_best_coherence)
        return best_coherence, Topic_best_coherence
    
    
    def main_func(self,inp1, inp2,num_topic,random_state, chunksize, passes):
        data_words,data_wordsnew = self.convert_to_list(inp1, inp2)
        
        corpus_o,id2word_o= self.data_processing(data_words)
        corpus_new,id2word_new= self.data_processing(data_wordsnew)
        # Training for the first time
        print("****TRAINING A LDA MODEL****")
        modelfit = self.Lda_fit(data_words,corpus_o,id2word_o,num_topic,random_state, chunksize, passes)
        self.coherence(data_words,id2word_o,modelfit)

        print("\n****TRAINING THE LDA MODEL FOR MULTIPLE NUMBER OF TOPICS****")
        model_list, coherence_values = self.compute_coherence_values(data_words,corpus_o,id2word_o)

        print('\n*****************************************************************************************************************************')
        print("\n****CALCULATING THE COHERENCE SCORE AND OPTIMAL NUMBER OF TOPICS FOR THE BEST MODEL OBTAINED****")
        best_coherence, Topic_best_coherence = self.best_model_lda(data_words,model_list, coherence_values)
        print('\n*****************************************************************************************************************************')
        #new model
        print('\n****PREDICTING NEW INSTANCES BASED ON BEST LDA MODEL****')
        self.Lda_fit(data_wordsnew,corpus_new,id2word_new,Topic_best_coherence,random_state, chunksize, passes)



   
  

In [12]:
A = fit_lda()
A.main_func(arr,arr1,5,20, 20, 10)

****TRAINING A LDA MODEL****
[(0,
  '0.012*"you" + 0.012*"whatever" + 0.012*"info" + 0.012*"car" + '
  '0.012*"neighborhood" + 0.012*"your" + 0.012*"by" + 0.012*"funky" + '
  '0.012*"on" + 0.012*"thanks"'),
 (1,
  '0.012*"this" + 0.012*"of" + 0.012*"is" + 0.012*"car" + 0.012*"the" + '
  '0.012*"anyone" + 0.012*"if" + 0.012*"engine" + 0.012*"years" + '
  '0.012*"model"'),
 (2,
  '0.060*"the" + 0.049*"was" + 0.037*"from" + 0.037*"car" + 0.025*"it" + '
  '0.025*"this" + 0.014*"were" + 0.014*"separate" + 0.014*"be" + '
  '0.014*"small"'),
 (3,
  '0.050*"this" + 0.035*"is" + 0.035*"of" + 0.035*"you" + 0.035*"car" + '
  '0.019*"know" + 0.019*"can" + 0.019*"tellme" + 0.019*"all" + '
  '0.019*"production"'),
 (4,
  '0.012*"was" + 0.012*"car" + 0.012*"sports" + 0.012*"anyone" + '
  '0.012*"looked" + 0.012*"day" + 0.012*"there" + 0.012*"door" + 0.012*"saw" + '
  '0.012*"could"')]
*********************************************************************************************************************